## Generación de texto mediante redes neuronales recurrentes

Un ejemplo montado en Keras para RNN usando una GRU con un texto del Quijote Shakespeare

# 1. Librerias

In [ ]:
from __future__ import absolute_import, division, print_function
from tensorflow import keras as ks
import tensorflow as tf

import numpy as np
import os
import time

!pip install unidecode
import unidecode

# 2. Cargamos los datos - el corpus basado en textos del Quijote

In [ ]:
def preprocess(text):
    """Receives a text and reduces the amount of symbols. Passes to lowercase.
    Args:
        text: input text to transform
    """
    text = unidecode.unidecode(text)
    text = text.replace('\n', ' ').replace('\ufeff', '').lower()
    text = ''.join(x for x in text if x not in "%$#=<>/*+@][")
    while len(text) != len(text.replace("  ", " ")):
        text = text.replace("  ", " ")
    return text


Preparamos los datos haciendo algunas manipulaciones, como la primera, decodificar el texto que viene en UTF-8

In [ ]:
with open("./quijote.txt", 'r') as infile:
  text = preprocess(infile.read())

print ('Longitud del corpus: {} caracteres'.format(len(text)))
print ('Ejemplo de texto...')
print(text[:250])
vocab = sorted(set(text))
print ('{} caracteres unicos'.format(len(vocab)))

# 3. Pre-proceso de los datos

## 3.1 Indexado de carácteres
Montamos un indice para los carácteres, para tener valores numéricos, y vemos un ejemplo

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')
print ('{} ---- carácteres mapeados a números enteros ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

Fijamos cada texto en secuencias de 100 carácteres, y mostrampos como se va a ir entregando la inforamción a la red neuronal recurrente...

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

## 3.2 Definimos las secuencias

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(idx2char[item.numpy()])
  print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(2):
  print ('Secuencia de entrada: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Secuencia de salida:  ', repr(''.join(idx2char[target_example.numpy()])))

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Paso {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  output esperado: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

## 3.3 Montamos el dataset

In [ ]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

# 4. Montamos la red neuronal recurrente

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = ks.Sequential()
    model.add(ks.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]))
    model.add(ks.layers.GRU(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True))
    model.add(ks.layers.Dense(vocab_size))
    
    return model

In [ ]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [ ]:
model.summary()

#5. Cogemos el modelo con pesos aleatorios que hemos creado y generamos una primera predicción con el mismo

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1): 
    print(input_example_batch)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

#6. Creamos una funcion de perdida loss

La función de pérdida estándar <code>tf.keras.losses.sparse_categorical_crossentropy</code> funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits, necesitamos establecer la <code>from_logits</code>. Logits son un conjunto de probabilidades sin scalar, es decir, que no hay que meter ningún "softmax" en la salida de la red neuronal.

Aprovechamos y calculamos el error escalar que estamos cometiendo en la predicción que hemos hecho en la anterior etapa

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

#7. Compilamos el modelo

In [ ]:
model.compile(
    optimizer = "adam",
    loss = loss,
    metrics = ['accuracy'])

#8. Definimos checkpoints donde almacenar los modelos a cada epoch, usando callbacks

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=ks.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

#9. Entrenamos el modelo 

In [ ]:
EPOCHS=10
history = model.fit(dataset.repeat(), epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

# 10. Reconstruimos el modelo con los pesos entrenados

Debido a la forma en que el estado RNN se pasa de un paso de tiempo a otro, el modelo solo acepta un tamaño de batch fijo una vez construido.

Para ejecutar el modelo con un batch_size diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el checkpoint.


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

#11. Montamos una función para generar texto

Consiste en que iteremos generando 1000 caracteres a partir de una semilla definida.

Existe un parametro llamado temperatura que lo modificaremos para ver los resultados.

In [ ]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 0.5
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

      if idx2char[predicted_id] in [",", "."]:
        text_generated.append("\n")

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"don quijote"))